# Проект "Компьютер говорит Нет"

### В этом соревновании необходимо предсказать допустит ли потенциальный заемщик дефолт.

В ходе реализации проекта:
- Создадим скоринговую модель предсказания дефолта клиентов банка.
- Поучаствуем в хакатоне.
- По доброй традиции обогатим своё портфолио!

### IMPORT

In [187]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics # инструменты для оценки точности модели

from imblearn.over_sampling import SMOTE

from sklearn import preprocessing

import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### DATA

In [188]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [189]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

### Функции предобработка

In [190]:
#Функция первичного осмотра данных. Вход: data - исследуемый датасет, brief_descr - краткое название датасета. Выход: void 
def first_look(data, brief_descr):
    print(brief_descr)
    print("Размерность: {}".format(data.shape))
    n = 5
    print("Первые {} строк".format(n))
    display(data.head(n))
    print("Детальная информация: ")
    data.info()
    #Проверим на пропуски
    for col in data.columns:
        if data[col].isnull().any():
            print("\n Признак {} содержит пропуски.".format(col))

# Определение min и max для числовых значений. На вход подаем имя столбца, на выходе [min,max]
def min_max(column):
    IQR = data[column].quantile(0.75) - data[column].quantile(0.25)
    minimum = data[column].quantile(0.25) - 1.5*IQR
    maximum = data[column].quantile(0.75) + 1.5*IQR
    return [minimum, maximum]


## Первичный осмотр и визуализация

In [191]:
#Образец данных на сабмишн
first_look(sample_submission,"Образец данных на сабмишн")

In [192]:
#Тестовый датасет
first_look(df_test,"Тестовый датасет")

In [193]:
#Тренировочный датасет
first_look(df_train,"Тестовый датасет")

Признак education содержит пропуски, которые необходимо будет в дальнейшем устранить.

Для корректной обработки признаков, сооберем тестовый и тренировочный датасеты в один.

In [194]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [195]:
first_look(data,'Объединенный датасет')

#### Краткое описание признаков.
- client_id - идентификатор клиента
- education - уровень образования
- sex - пол заемщика
- age - возраст заемщика
- car - флаг наличия автомобиля
- car_type - флаг автомобиля иномарки
- decline_app_cnt - количество отказанных прошлых заявок
- good_work - флаг наличия “хорошей” работы
- bki_request_cnt - количество запросов в БКИ
- home_address - категоризатор домашнего адреса
- work_address - категоризатор рабочего адреса
- income - доход заемщика
- foreign_passport - наличие загранпаспорта
- sna - связь заемщика с клиентами банка
- first_time - давность наличия информации о заемщике
- score_bki - скоринговый балл по данным из БКИ
- region_rating - рейтинг региона
- app_date - дата подачи заявки
- default - флаг дефолта по кредиту

Взглянем более детально на признаки.

In [196]:
data.nunique(dropna=False)

Признаки 'car_type', 'foreign_passport', 'car','good_work','sex' являются бинарным.

Признаки 'decline_app_cnt','income','age','score_bki', 'bki_request_cnt' относятся к количественным.

Категориальные признаки: 'first_time', 'sna', 'work_address', 'home_address', 'region_rating'.

Признак 'education' можно отнести к ординальным. В этом признаке присутствует отношение порядка: школа является более низкой ступенью по отношению к университету, а, например, докторская степень к бакалавру.

In [197]:
#Количественные переменные
num_cols = [ 'decline_app_cnt','income','age','score_bki', 'bki_request_cnt']

#Категориальные переменные
cat_cols = [ 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']

#Бинарные категориальные переменные
bin_cols = [ 'car_type', 'foreign_passport', 'car','good_work','sex']#

#Ординальные переменные
ord_cols = ['education']

Взглянем на целевую переменную.

In [198]:
data['default'].value_counts(ascending=True).plot(kind='barh')
print("Баланс классов целевой переменной")
print(data['default'].value_counts()/data['default'].shape[0])

Выборка сильно несбалансирована. Соотношение классов примерно 1 к 9 в пользу надежных заемщиков. Дисбаланс классов может стать причиной проблем при обучении модели.

### Очистка данных

Столбцы 'client_id','app_date' не несут полезной нагрузки. Поэтому удаляем их.

In [199]:
data.drop(['client_id','app_date'], axis = 1, inplace=True)

Переведем значения бинарных столбцов из буквенных в численные значения.

In [200]:
le = preprocessing.LabelEncoder()

for column in bin_cols:
    data[column] = le.fit_transform(data[column])
    
columns = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']

for column in columns:
    data[column] = le.fit_transform(data[column])

### Заполнение пропусков
Для того, чтоб заполнить пропуски в переменной education, воспользуемся методом k-ближайших соседей.
Признак education можно считать ординальным, так как он отражает ступени образования.
- SCH - общая школа
- URG - бакалавры
- GRD - высшая школа
- PGR - аспиратнтура, кандидатская
- ACD - академия, докторская
- UNK - пропуск данных

In [201]:
#заполняем пропуски значением UNK
data['education'].fillna('UNK', inplace=True)

#словарь замены значений Education в числовые
dict_replace = {"UGR": 2, "SCH": 1, "GRD" : 3, "PGR": 4, "ACD": 5, "UNK": -1}

#меняем значения на числовые
data['education'] = data['education'].apply(lambda x: dict_replace[x])

In [202]:
#Выделим данные без пропусков для обучения и данные с пропусками для последующего заполнения
data_train = data[data.education > -1].copy()
data_edu_void = data[data.education == -1].copy()

#Выделим обучабщие признаки и таргет. Из предикторов исключим "education", как целевой,'sample' как неинформативный, 
#default - для избежания загрязнения целевой переменной.
X1 = data_train.drop(columns=['education','sample','default'])
Y1 = data_train['education']

In [203]:
#создадим модель kNN и обучим ее
model_knn = KNN(n_neighbors=5)
model_knn.fit(X1,Y1)

#Заменим пропущенные значение, предсказанными моделью 
data_edu_void['education'] = model_knn.predict(data_edu_void.drop(columns=['education','default','sample']))

#Заменим пропуски (-1) в исходном датасете, на полученные из модели.
data['education'].update(data_edu_void['education'])

In [204]:
data.info()

Мы получили датасет, заполненный исключительно числовыми значениями. Теперь можно построить наивную модель для получения базовой метрики, на которую в дальнейшем можно будет ориентироваться.

## НАИВНАЯ МОДЕЛЬ

In [205]:
# Выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создадим экземпляр модели логистической регрессии без каких-либо параметров, кроме max_iter.

In [206]:
logreg_naive = LogisticRegression(max_iter=1000)
logreg_naive.fit(X_train, y_train)
y_pred = logreg_naive.predict(X_test)

Выведем отчет о метриках. Нас будет интересоваться F1 на классе 1.

In [207]:
cl_rp = classification_report(y_test, y_pred)
print(cl_rp)

In [208]:
#Строим confusion matrix
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues')
ax.set_title('Seaborn Confusion Matrix with labels')
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual Values ')
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])
plt.show()

Из-за сильного дисбаланса классов, а также неподготовленных данных, модель абсолютно бесполезна - она всегда предсказывает, что клиент не является дефолтным.

## Разведывательный анализ

### Визуализация
Построим графики распределения количественных переменных.

In [209]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Можно увидеть, что большая часть переменных имеет выраженный правый хвост.
Прологарифируем количественные значения для того, чтоб сделать распределения более нормальными. Кроме score_bki, который уже и так вполне себе "колокол" (похож на нормальное распределение).

In [210]:
#Логирифмируем переменные и строим новые графики
for i in num_cols:
    if i == 'score_bki': continue
        
    data[i] = data[i].apply(lambda x: np.log(abs(x)+1))
    plt.figure()
    sns.distplot(data[i][data[i] > 0], kde = False, rug=False)
    plt.title(i)
    plt.show()

### Матрица корреляций
Построим матрицу корреляций между количественными признаками

In [211]:
sns.heatmap(data[num_cols+ord_cols].corr().abs(), vmin=0, vmax=1)

Средняя корреляция наблюдается между образованием и уровнем дохода. Но это вполне естественно.

В остальном сильных взаимосвязей между непрерывными переменными нет. Удалять ничего не приедтся.

### Значимость количественных переменных

Оценим значимость количественных переменных с помощью f_classif.

In [212]:
imp_num = pd.Series(f_classif(data[num_cols+ord_cols], data['default'])[0], index = num_cols + ord_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')
display(imp_num)

Судя по отчету, возраст и доход оказывают слабое влияние на вероятность дефолта. Можно будет попробовать убрать малозначимые признаки при поиске оптимального набора в финальный датасет.

### Значимость категориальных переменных

Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn.

In [213]:
imp_cat = pd.Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')
display(imp_cat)

Похоже, что пол, наличие хорошей работы и автомобиля не являются определяющими.

### Поиск и устранение вылетов

Ввиду того, что логистическая регрессия чувствительна к вылетам, необходимо произвести их поиск и устранение.

Для начала построим боксплоты.

In [214]:
for i in num_cols:
    data[[i]].boxplot()
    plt.show()

Кроме признака age, все остальные содержат аномальные выбросы.

Рассмотрим каждый признак отдельно.

**decline_app_cnt**

In [215]:
data['decline_app_cnt'].value_counts()

Признак можно перекодировать в ординальный. 0 = 0; значения больше ноля, но меньше 1 = 1; все остальное 2.

In [216]:
data['decline_app_cnt'] = data['decline_app_cnt'].apply(lambda x: 0 if x == 0 else 1 if 0 < x < 1 else 2)
num_cols.remove('decline_app_cnt')
ord_cols.append('decline_app_cnt')

**bki_request_cnt**



In [217]:
data['bki_request_cnt'].value_counts()

Перекодируем признак в ординальный с разбиением по квартилям 20%, 40%, 60% и 80%.

In [218]:
quantiles = []
for q in [0.2,0.4,0.6,0.8]:
    quantiles.append(data['bki_request_cnt'].quantile(q))
    
data['bki_request_cnt'] = data['bki_request_cnt'].apply(lambda x: 0 if x <= quantiles[0] else 1 if quantiles[0] < x <= quantiles[1] else 
                                                       2 if quantiles[1] < x <= quantiles[2] else 3 if quantiles[2] < x <= quantiles[3] else 4)
   
num_cols.remove('bki_request_cnt')
ord_cols.append('bki_request_cnt')

**score_bki, income**

Заменим вылеты в этих признаках на пограничные значения 1,5 IQR.

In [219]:
for col in num_cols:
    #Получаем минимальные и максимальные значения 1,5*IQR
    minimum,maximum = min_max(col)
    
    data[col] = data[col].apply(lambda x: minimum if x < minimum else maximum if x > maximum else x)

Проверим, что получилось после устранения вылетов.

In [220]:
for i in num_cols:
    data[[i]].boxplot()
    plt.show()

## Преобразование переменных

In [221]:
#Стандартизируем числовые переменные
Scaler = StandardScaler()
Scaler.fit(data[num_cols])
data[num_cols] = Scaler.transform(data[num_cols].values)


In [222]:
#Преобразуем категориальные переменные с помощью OneHotEncoder
X_cat = OneHotEncoder(sparse = False).fit_transform(data[cat_cols])
data = pd.concat([data,pd.DataFrame(X_cat)],axis=1)

#Удаляем преобразованные категориальные переменные из датасета
data.drop(columns=cat_cols, inplace=True)

Получили подготовленный датасет.

In [223]:
data.info()

## Построение моделей

Данные обработаны. Теперь необходимо построить модель и поэкспериментировать.

In [224]:
#Добавляем фиктивный столбец с единицами
data['ones'] =  pd.Series(np.ones(data.shape[0]))

In [225]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [226]:
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [227]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [228]:
#Строим модель
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [229]:
#Выводим метрики.
cl_rp = classification_report(y_test, y_pred)
print(cl_rp)


Показатель F1 для класса 1 стал 0.04 (был 0). Прогресс хоть и небольшой, но есть. Попробуем определить гиперпараметры, для повышения точности.

### Гиперпараметры.

In [230]:
# запускаем GridSearch на небольшом кол-ве итераций max_iter=50 и с достаточно большой дельтой останова tol1e-3
# чтобы получить оптимальные параметры модели в первом приближении
model = LogisticRegression()

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':[None, 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':[None, 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':[None, 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

    
## model - модель логистической регрессии
gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_


In [231]:
##печатаем параметры
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [232]:
#Обучаем модель с полученными гиперпараметрами.
logreg = LogisticRegression(C = 1.0, class_weight = 'balanced', dual = False, fit_intercept = True, intercept_scaling = 1,
                            l1_ratio = None, multi_class = 'auto', n_jobs = None, penalty = "l2", tol = 0.001, 
                            max_iter=1000,solver='newton-cg', warm_start = False)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

#Выводим метрики
cl_rp = classification_report(y_test, y_pred)
print(cl_rp)

Метрика F1 для класса 1 выросла до 0.34, учитывая дисбаланс классов, не так уж и плохо.

### Оверсемплинг

В датасете наблюдается дисбаланс классов. Попробуем с помощью метода SMOTE синтезировать экземпляры класса 1 для того, чтоб уравнять классы.

In [233]:
#Попробуем оверсемплинг на тренировочном наборе
sm = SMOTE(random_state=42)
X_train_samp, y_train_samp = sm.fit_resample(X_train, y_train)

In [234]:
#Посмотрим на баланс классов после оверсемплинга
class_balance = pd.Series(y_train_samp).value_counts()/pd.Series(y_train_samp).shape[0]
print(class_balance)

In [235]:
#Обучаем модель на оверсемплированном датасете
logreg_samp = LogisticRegression(C = 1.0, class_weight = 'balanced', dual = False, fit_intercept = True, intercept_scaling = 1,
                            l1_ratio = None, multi_class = 'auto', n_jobs = None, penalty = "l2", tol = 0.001, 
                            max_iter=1000,solver='newton-cg', warm_start = False)
logreg_samp.fit(X_train_samp, y_train_samp)
y_pred = logreg_samp.predict(X_test)

In [236]:
print(classification_report(y_test,y_pred))

Оверсемплинг ухудшил целевую метрику. Применять его не имеет смысла.

### Удаление малозначимых признаков

Попробуем удалить малозначимые количественные и категориальные признаки из датасета и посмотрим, как это повлияет на целевую метрику.

Среди количественных малозначимыми оказались 'age' и 'income', среди категориальных - 'sex', 'car', 'good_work'.

In [237]:
#Список признаков на исключение
rem_cols = ['age','income','sex','car','good_work']

#модель для проверки удаления малозначимых признаков
logreg_insign_remove = LogisticRegression(C = 1.0, class_weight = 'balanced', dual = False, fit_intercept = True, intercept_scaling = 1,
                            l1_ratio = None, multi_class = 'auto', n_jobs = None, penalty = "l2", tol = 0.001, 
                            max_iter=1000,solver='newton-cg', warm_start = False)

#Прогоним модель поочередно исключая каждый из малозначимых признаков + в конце исключим все.
for col in rem_cols:
    #Удаляем признак
    X_insign_rem_train = X_train.drop(columns=[col])
    X_insign_rem_test = X_test.drop(columns=[col])
    
    #Обучаем модель
    logreg_insign_remove.fit(X_insign_rem_train,y_train)
    y_isnign_pred = logreg_insign_remove.predict(X_insign_rem_test)
    
    print("Удален признак {}. F1-score: {}".format(col,
                                                    round(f1_score(y_test,y_isnign_pred),4)))

#Пробуем удалить все признаки сразу.
X_insign_rem_train = X_train.drop(columns=rem_cols)
X_insign_rem_test = X_test.drop(columns=rem_cols)
    
#Обучаем модель
logreg_insign_remove.fit(X_insign_rem_train,y_train)
y_isnign_pred = logreg_insign_remove.predict(X_insign_rem_test)
    
print("Удален признак {}. F1-score: {}".format(rem_cols,
                                                round(f1_score(y_test,y_isnign_pred),4)))

Удаление признаков не оказывает положительного эффекта на целевую метрику.

## Оценка модели

Лучшим показателем F1 обладает модель, сохраняющая все признаки для обучения и применяющая гиперпараметры. Ее и будем считать финальной.

Итоговый показатель метрики F1 составляет 0.34, что весьма мало, но, вероятно, является близким к пределу ввиду сильной несбалансированности классов в исходном датасете, а также сильной схожести классов ("смешанности классов").

Также за кадром были опробованы генерация полиномиальных признаков - она не дала прироста метрики; отключение обработки выбросов количественных признаков давало прирост 0,002 к метрике F1, но ввиду того, что мы используем L2-регуляризацию, выбросы в реальных условиях могут оказать отрицательное влияние и снизить качество модели, поэтому их обработку решено было оставить.

## Отправка результатов

In [238]:
# если качество нас устраивает, обучаем финальную модель на всех обучающи данных
logreg_final = LogisticRegression(C = 1.0, class_weight = 'balanced', dual = False, fit_intercept = True, intercept_scaling = 1,
                            l1_ratio = None, multi_class = 'auto', n_jobs = None, penalty = "l2", tol = 0.001, 
                            max_iter=1000,solver='newton-cg', warm_start = False)
logreg_final.fit(X, y)

In [239]:
predict_submission = logreg_final.predict(test_data)

In [240]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [241]:
sample_submission.describe()

In [242]:
!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'